In [1]:
# batchnorm (O)
# random mask=> random sampling
# std 有點大 around 10 > normalize to 1 (0)
# mean around [-1,1], maybe we can abs() => mean to 0 (O)
# # of FC layers <5, 遞減 256 64 16 4 (O)

In [2]:
import numpy as np
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

In [3]:
# Turn the data into 0 mean and 1 var
from sklearn import preprocessing
for i in range(X_train_valid.shape[0]):
        X_train_valid[i] = preprocessing.scale(X_train_valid[i])
for i in range(X_test.shape[0]):
    X_test[i] = preprocessing.scale(X_test[i])

In [4]:
# Not use the last 3 of the 25 electrodes, which are EOG (rather than EEG) electrodes
X_train_valid = X_train_valid[:,:-3,:]
X_test = X_test[:,:-3,:]

# Change the timestep to the second column
X_train_valid = np.transpose(X_train_valid, (0, 2, 1))
X_test = np.transpose(X_test, (0, 2, 1))

# Modify the y to categorical form
from keras.utils import np_utils
num_classes = np.unique(y_train_valid).size
y_train_valid = y_train_valid - min(y_train_valid)
y_test = y_test - min(y_test)
y_train_valid = np_utils.to_categorical(y_train_valid, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


In [5]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 1000, 22)
Test data shape: (443, 1000, 22)
Training/Valid target shape: (2115, 4)
Test target shape: (443, 4)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [6]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras import optimizers
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt

%matplotlib inline

In [7]:
# Hyper parameters
batch_size = 64
epochs = 1
dropout_prob = 0.5

# Parameters for LSTM network
lstm_outputs = 256
_, timesteps, data_dim = X_train_valid.shape

In [8]:
optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7, decay=0.99)

In [9]:
model = Sequential()
model.add(LSTM(lstm_outputs, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension lstm_outputs
model.add(BatchNormalization())
model.add(Dropout(dropout_prob))
model.add(LSTM(lstm_outputs, return_sequences=True))  # returns a sequence of vectors of dimension lstm_outputs
model.add(BatchNormalization())
model.add(Dropout(dropout_prob))
model.add(LSTM(lstm_outputs))  # return a single vector of dimension 32
model.add(BatchNormalization())
model.add(Dropout(dropout_prob))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout_prob))
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout_prob))
model.add(Dense(num_classes, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1000, 256)         285696    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1000, 256)         1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 256)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 1000, 256)         525312    
_________________________________________________________________
batch_normalization_2 (Batch (None, 1000, 256)         1024      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000, 256)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
__________

In [11]:
history = model.fit(X_train_valid, y_train_valid, batch_size=batch_size, 
                    epochs=epochs, validation_split = 0.1, shuffle=True)

Train on 1903 samples, validate on 212 samples
Epoch 1/1
1903/1903 [==============================] - 549s - loss: 2.1639 - acc: 0.2517 - val_loss: 1.3867 - val_acc: 0.2406


In [14]:
#### get predicted action for each signal in test set
predictions = model.predict(X_test)
# print out test accuracy
test_accuracy = 100*np.sum(np.argmax(predictions,axis =1)==np.argmax(y_test, axis=1))/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 23.0248%
